In [ ]:
import os
import requests
import json
import re
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

base_dir = "partiallyd-scrape"
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

In [ ]:
url = "http://partiallyderivative.com/podcast/"
podcast_page = BeautifulSoup(requests.get(url).text, "lxml")

In [ ]:
raw_urls = []
raw_titles = []
raw_dates = []

post_list = podcast_page.find('ul', {'class':'post-list'})
for li in post_list.findAll('li'):
    link = li.find('a')
    span = li.find('span')
    raw_dates.append(span.text)
    raw_urls.append(link['href'])
    name = li.text[:-len(span.text)]
    raw_titles.append(name)

In [ ]:
links = { }
for raw_url in raw_urls:
    podcast_article = BeautifulSoup(requests.get(raw_url).text, "lxml")
    ol = podcast_article.find('ol')
    links[raw_url] = []
    if ol is None:
        continue
    for li in ol.findAll('li'):
        a = li.find('a')
        try:
            links[raw_url].append([a.text, a['href']])
        except:
            print(raw_url)

In [ ]:
reg = re.compile('([0-9]{4})/([0-9]{2})/([0-9]{2})/([a-zA-Z0-9-_]+)')
dates = []
slug = []
count = []

for u in raw_urls:
    match = reg.search(u)
    if match:
        day = int(match.group(3))
        month = int(match.group(2))
        year = int(match.group(1))
        dt_str = "%04d-%02d-%02d" % (year,month,day)
        try:
            slug.append(match.group(4))
            dates.append(pd.to_datetime(dt_str))
            count.append(len(links[u]))
        except:
            print(dt_str)
podcasts_df = pd.DataFrame({
    'id': slug,
    'date': dates,
    'count': count
})

podcasts_df = podcasts_df.set_index('date')

print(podcasts_df.head())
podcasts_df.to_csv("partiallyd-scrape/podcasts.csv")

In [ ]:
podcasts_df.plot(figsize=(15,4))
plt.show()

In [ ]:
podcasts_w_links = podcasts_df[podcasts_df['count'] != 0]
podcasts_w_links.plot(figsize=(15,4))
plt.show()

In [ ]:
podcasts_w_links.describe()